In [15]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1283, in

In [20]:
# from urllib.request import urlopen
# with urlopen("https://jupyter-33929658-132a-458e-9730-8168389c331e.lambdaspaces.com/lab/tree/IMDB_train") as response:
#     body = response.read()

In [13]:
import torch, os, re, pandas as pd, json
import torch
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig

import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import urllib2

ModuleNotFoundError: No module named 'urllib2'

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
# base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')

In [5]:
head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to(device)

In [6]:
import os

pos_path = "/home/ubuntu/IMDB_train/pos/"
neg_path = "/home/ubuntu/IMDB_train/neg/"

pos_token_list = []
pos_text_list = []
neg_token_list = []
neg_text_list = []

os.chdir(pos_path)
  

for file in os.listdir():
    if not file.endswith('.txt'):
        continue
    with open(pos_path+file, 'r') as f:
        pos_text = f.read()
        # put into text list
        pos_text_list.append(pos_text)
        #tokenize and put into token list
        #pos_token_list.append(base_tokenizer.encode(pos_text, return_tensors = 'pt'))

os.chdir(neg_path)

for file in os.listdir():
    if not file.endswith('.txt'):
        continue
    with open(neg_path+file, 'r') as f:
        neg_text = f.read()
        # put into text list
        neg_text_list.append(neg_text)
        #tokenize and put into token list
        #neg_token_list.append(base_tokenizer.encode(neg_text_list, return_tensors = 'pt'))

In [7]:
# df_pos_tokens = pd.DataFrame(pos_token_list, columns = ['tokens'])
# print(df_pos_tokens)
df_pos_text = pd.DataFrame(pos_text_list, columns = ['text'])
print(df_pos_text)

                                                    text
0      Legend of Dragoon is one of those little-known...
1      to communicate in film essential things of lif...
2      Excellent episode movie ala Pulp Fiction. 7 da...
3      I saw this in theaters and absolutely adored i...
4      At the time I am writing this I see out of ove...
...                                                  ...
12495  The Ring was made from the only screenplay Hit...
12496  This is a very strange HK film in many ways. F...
12497  With the current trend of gross out humor, thi...
12498  This movie i've loved since i was young! Its e...
12499  An American in Paris is a showcase of Gene Kel...

[12500 rows x 1 columns]


In [8]:
text = "I work as a data scientist"
text_ids = base_tokenizer.encode(text, return_tensors = 'pt')
print(text_ids.shape)
print(head_model(text_ids.to(device)).logits.shape)

torch.Size([1, 6])
torch.Size([1, 6, 50257])


In [9]:
class ImdbPos(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2-xl", max_length=1024):

        self.tokenizer = base_tokenizer # can change
        self.text = []

        for row in df_pos_text['text']:
            self.text.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        if truncate:
            self.text = self.text[:20000]
        self.text_count = len(self.text)
        
    def __len__(self):
        return self.text_count

    def __getitem__(self, item):
        return self.text[item]
    
dataset = ImdbPos(df_pos_text['text'], truncate=True, gpt2_type="gpt2-xl")  

In [10]:
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [11]:
def train(
    dataset, model, tokenizer,
    batch_size=10, epochs=20, lr=2e-5,
    max_seq_len=1000, warmup_steps=200,
    gpt2_type="gpt2-xl", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    # device=torch.device("cuda")
    model = model.to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=1250*20
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor
            outputs = model(input_tensor.to(device), labels=input_tensor.to(device))
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [12]:
# pos_imdb_model = train(dataset, head_model, base_tokenizer, epochs = 20,  max_seq_len=1000, batch_size=16)
pos_imdb_model = train(dataset, head_model, base_tokenizer)

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


287it [00:22, 12.89it/s]


RuntimeError: CUDA out of memory. Tried to allocate 146.00 MiB (GPU 0; 39.41 GiB total capacity; 36.34 GiB already allocated; 106.56 MiB free; 37.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF